## **Import Libraries**

In [7]:
import torch
from torch_geometric.nn import GCNConv

## **Load the Trained Model**

convert this class to a .py script later on!

In [8]:
class SimpleGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)  # First GCN layer
        self.conv2 = GCNConv(16, num_classes)        # Second GCN layer

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x  # No log_softmax for regression

In [9]:
num_node_features = 4  # From your Data object, x=[713, 4]
num_classes = 1  # Assuming a regression task; adjust based on your specific task

model = SimpleGCN(num_node_features, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Using Adam optimizer

In [10]:
# paste specific model path
model_iteration = 'model_2024-02-15_22-14-57'

In [12]:
# Load the saved state_dict
model.load_state_dict(torch.load(f'pytorch_model/{model_iteration}.pth'))

<All keys matched successfully>

In [13]:
# Ensure you call model.eval() to set dropout and batch normalization layers to evaluation mode
model.eval()

SimpleGCN(
  (conv1): GCNConv(4, 16)
  (conv2): GCNConv(16, 1)
)